In [1]:
import multiprocessing
multiprocessing.set_start_method('fork')


In [2]:
import collections
import random
from random import  uniform
from math import *
import scipy.optimize as opt

def H(c):
    """
    Entropy function
    """
    if c == 0. or c == 1.:
        return 0.
    
    if c < 0. or c > 1.:
        return -1000
    
    return -(c * log2(c) + (1 - c) * log2(1 - c))

def binomH(n,k):
    """
    binomial coefficient
    """
    # if k/n not in ZZ:
    #     return -100
    if(n<=0):
        return 0.
    return n * H(k/n)

def multiH(n,c):
    """
    multinomial coefficient
    """
    if sum(c)>n:
        return 0
    tot=0
    val=n
    for i in c:
        tot+=binomH(n,i)
        n-=i
    return tot


def wrap(f,g) :
    def inner(x):
        return f(g(*x))
    return inner

def r(x,y,z):
    return [(ru(x,y)) for i in range(z)]

In [3]:
# Nested Collision Search

def rep3(n1,n2,n3,z1,z2,z3,o1,o2,t,t1,r,l): 
    """
    representations of length-l vector in Rep-3 that has n1 many \pm1, n2 many \pm2, n3 many \pm3 each.
    """
#     if n1 <= 0.00001 or l == 0. or n2 <= 0.00001 or n3 <= 0.00001:
#         return 0
#     if n1 <= 0.00001 or l == 0.:
#         return 0
    m = l-2*(n1+n2+n3+z1+z2+z3)
    R0 = multiH(l-2*(n1+n2+n3),[m,z1,z1,z2,z2,z3])
    R1 = multiH(n1,[n1*.5-o1-o2,n1*.5-o1-o2,o1,o1,o2])
    R2 = multiH(n2,[n2*.5-t*.5-t1,n2*.5-t*.5-t1,t,t1])
    R3 = multiH(n3,[n3*.5-r,n3*.5-r,r])
    return R0+2*(R1+R2+R3)


set_vars = collections.namedtuple('LWE', ' mz1 mz2 mz3 mo1 mo2 mt mt1 mr z1 z2 z3 o1 o2 t t1 r g b1 b2 b3')

"""
The "m" in the beginning of the name of the variable denotes that the variable is for top to mid level,
"g" denotes the gamma parameter,
the other variables are for mid to base level. b1, b2, b3 are for permuting the \pm1s, \pm2s, pm3s respectively.
"""

def lwe(f) : return wrap(f, set_vars)


#[w1(x),w2(x),w3(x)] = [1/3,0,0]            #weight distribution for U(1)

#[w1(x),w2(x),w3(x)] = [1/5,1/5,0]          #weight distribution for U(2)

#w1(x),w2(x),w3(x) = [1/7,1/7,1/7]        #weight distribution for U(3)

#[w1(x),w2(x),w3(x)] = [15/64,6/64,1/64]    #weight distribution for B(3)


# Number of \pm1, \pm2, \pm3 in the \gamma n part of the final solution
n1 = lambda x : w1(x)*x.g*x.b1
n2 = lambda x : w2(x)*x.g*x.b2
n3 = lambda x : w3(x)*x.g*x.b3
n0 = lambda x : x.g - 2*(n1(x)+n2(x)+n3(x))


# Number of \pm1, \pm2, \pm3 in the \gamma n part of the mid level summands.  "nm" denotes numbers in mid level
nm1 = lambda x : x.mz1+n1(x)*.5-x.mo2+x.mt+x.mt1+x.mr
nm2 = lambda x : x.mz2+x.mo1+x.mo2+n2(x)*.5-x.mt*.5-x.mt1+x.mr
nm3 = lambda x : x.mz3+x.mo2+x.mt1+n3(x)*.5-x.mr
nm0 = lambda x : x.g - 2*(nm1(x)+nm2(x)+nm3(x))   

# Number of \pm1, \pm2, \pm3 in the \gamma n part of the base level summands. "nb" denotes numbers in base level
nb1 = lambda x : x.z1+nm1(x)*.5-x.o2+x.t+x.t1+x.r
nb2 = lambda x : x.z2+x.o1+x.o2+nm2(x)*.5-x.t*.5-x.t1+x.r
nb3 = lambda x : x.z3+x.o2+x.t1+nm3(x)*.5-x.r
nb0 = lambda x : x.g - 2*(nb1(x)+nb2(x)+nb3(x))   



def time(x):
    x=set_vars(*x)
    good_D = multiH(1,[w1(x),w1(x),w2(x),w2(x),w3(x),w3(x)]) - ss(x)
    good_R = max(0,domain(x)-rep3(n1(x),n2(x),n3(x),x.mz1,x.mz2,x.mz3,x.mo1,x.mo2,x.mt,x.mt1,x.r,x.g))
    collisions_when_R_good = domain(x) - 2*rep3(nm1(x),nm2(x),nm3(x),x.z1,x.z2,x.z3,x.o1,x.o2,x.t,x.t1,x.r,x.g)
    one_collision=domain(x)
    return good_R+collisions_when_R_good+one_collision+good_D

domain = lambda x : multiH(x.g,[nb1(x),nb1(x),nb2(x),nb2(x),nb3(x),nb3(x)]) \
+ multiH((1-x.g)*.25,[(1-x.g*x.b1)*.25*w1(x),(1-x.g*x.b1)*.25*w1(x),(1-x.g*x.b2)*.25*w2(x),(1-x.g*x.b2)*.25*w2(x),(1-x.g*x.b3)*.25*w3(x),(1-x.g*x.b3)*.25*w3(x)])

ss = lambda x : 4*multiH((1-x.g)*.25,[(1-x.g*x.b1)*.25*w1(x),(1-x.g*x.b1)*.25*w1(x),(1-x.g*x.b2)*.25*w2(x),(1-x.g*x.b2)*.25*w2(x),(1-x.g*x.b3)*.25*w3(x),(1-x.g*x.b3)*.25*w3(x)]) \
+ multiH(x.g,[x.g*x.b1*w1(x),x.g*x.b1*w1(x),x.g*x.b2*w2(x),x.g*x.b2*w2(x),x.g*x.b3*w3(x),x.g*x.b3*w3(x)])

W_disjoint = lambda x : ((1-x.g*x.b1)*w1(x)+(1-x.g*x.b2)*w2(x)+(1-x.g*x.b3)*w3(x))*.5

def set_initial_constraints():
    global constraints_lwe
    constraints_lwe = [
    # domain must be equal to 1/2
    { 'type' : 'ineq',   'fun' : lwe(lambda x : multiH(1,[w1(x),w1(x),w2(x),w2(x),w3(x),w3(x)]) - ss(x))},

    { 'type' : 'eq',   'fun' : lwe(lambda x : ss(x)*.5-domain(x))},

    { 'type' : 'ineq',   'fun' : lwe(lambda x : domain(x)-rep3(n1(x),n2(x),n3(x),x.mz1,x.mz2,x.mz3,x.mo1,x.mo2,x.mt,x.mt1,x.r,x.g))},

    { 'type' : 'ineq',   'fun' : lwe(lambda x : domain(x) - 2*rep3(nm1(x),nm2(x),nm3(x),x.z1,x.z2,x.z3,x.o1,x.o2,x.t,x.t1,x.r,x.g))},

    { 'type' : 'ineq',   'fun' : lwe(lambda x : nm1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : nm2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : nm3(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : nm0(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : nb1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : nb2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : nb3(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : nb0(x))},

    { 'type' : 'ineq',   'fun' : lwe(lambda x : n1(x)*.5-x.mo1-x.mo2)},

    { 'type' : 'ineq',   'fun' : lwe(lambda x : nm1(x)*.5-x.o1-x.o2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : nm2(x)*.5-x.t*.5-x.t1)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : nm3(x)*.5-x.r)},

    { 'type' : 'ineq',   'fun' : lwe(lambda x : x.g-2*(n1(x)+n2(x)+n3(x)+x.mz1+x.mz2+x.mz3))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : x.g-2*(nm1(x)+nm2(x)+nm3(x)+x.z1+x.z2+x.z3))},

    { 'type' : 'ineq',   'fun' : lwe(lambda x : (1-x.g)*.25-W_disjoint(x))},
    ]

def use_b1():
    global w1,w2,w3, constraints_lwe
    w1 = lambda x: 1/4
    w2 = lambda x: 0
    w3 = lambda x: 0
    extra = [{ 'type' : 'eq',   'fun' : lwe(lambda x : x.mr)},
             { 'type' : 'eq',   'fun' : lwe(lambda x : x.mt1)},
             { 'type' : 'eq',   'fun' : lwe(lambda x : x.mt)},]
    for i in extra:
        constraints_lwe.append(i)
        
def use_u1():
    global w1,w2,w3, constraints_lwe
    w1 = lambda x: 1/3
    w2 = lambda x: 0
    w3 = lambda x: 0
    extra = [{ 'type' : 'eq',   'fun' : lwe(lambda x : x.mr)},
             { 'type' : 'eq',   'fun' : lwe(lambda x : x.mt1)},
             { 'type' : 'eq',   'fun' : lwe(lambda x : x.mt)},]
    for i in extra:
        constraints_lwe.append(i)

def use_b2():
    global w1,w2,w3, constraints_lwe
    w1 = lambda x: 4/16
    w2 = lambda x: 1/16
    w3 = lambda x: 0
    extra = { 'type' : 'eq',   'fun' : lwe(lambda x : x.mr)}
    constraints_lwe.append(extra)
    
def use_u2():
    global w1,w2,w3, constraints_lwe
    w1 = lambda x: 1/5
    w2 = lambda x: 1/5
    w3 = lambda x: 0
    extra = { 'type' : 'eq',   'fun' : lwe(lambda x : x.mr)}
    constraints_lwe.append(extra)
        

def use_u3():
    global w1,w2,w3, constraints_lwe
    w1 = lambda x: 1/7
    w2 = lambda x: 1/7
    w3 = lambda x: 1/7
    extra = [{ 'type' : 'ineq',   'fun' : lwe(lambda x : n2(x)*.5-x.mt*.5-x.mt1)}, 
             { 'type' : 'ineq',   'fun' : lwe(lambda x : n3(x)*.5-x.mr)}]
    for i in extra:
        constraints_lwe.append(i)

def use_b3():
    global w1,w2,w3, constraints_lwe
    w1 = lambda x: 15/64
    w2 = lambda x: 6/64
    w3 = lambda x: 1/64
    extra = [{ 'type' : 'ineq',   'fun' : lwe(lambda x : n2(x)*.5-x.mt*.5-x.mt1)}, 
             { 'type' : 'ineq',   'fun' : lwe(lambda x : n3(x)*.5-x.mr)}]
    for i in extra:
        constraints_lwe.append(i)
        
def search_space():
    return multiH(1,[w1(x),w1(x),w2(x),w2(x),w3(x),w3(x)])

def uni(x,y,z):
    L=[]
    for i in range(z):
        L.append(uniform(x,y))
    return L

def opti():
    bounds=[(0,0.1)]*16+[(0,1)]*4
    res = 100
    ress = 5000
    for j in range(100):
        start = uni(0,0.0009,16)+uni(0.1,1,4)
        result = opt.minimize(time, start, 
                bounds= bounds, tol=1e-10, 
                constraints=constraints_lwe, options={'maxiter':1000})
        r = result.get('fun')
        if(r < res and r >0 and result.get('message') == 'Optimization terminated successfully'):
            res = r
            ress = result
    #print(round(res,4), ress)
    #print(ress.x)
    return [res,ress]


set_initial_constraints()



In [4]:
def do_parallel():
    mini=[1000]
    prev=1000
    for k in range(10):
        
        candidates=[]
        with multiprocessing.Pool(PROCESSES) as pool:
            params = range(PROCESSES)
            results = [pool.apply_async(opti) for p in range(PROCESSES)]
            for i in results:

                candidates.append(i.get())

        for i in candidates:
            if mini[0]==1000 or i[0]<mini[0]:
                mini=i
        
        if mini != prev:
            print(k,mini)
            prev=mini
    return mini





In [12]:
#### SPECIFY WHICH DISTRIBUTION TO USE  ####

use_b3()

#### SPECIFY NUMBER OF PROCESSES TO USE ####

PROCESSES=256

############################################

res,ress=do_parallel()
print("\n\n\n Best result found:\n")
print(round(res,4), ress)
print(ress.x) 

0 [1.1841630673762762,      fun: 1.1841630673762762
     jac: array([-1.76564021e+00, -1.06635480e+01, -1.68484333e+01, -1.06636586e+01,
       -2.57475477e+01,  3.56589538e+00, -7.95117863e+00, -6.74490575e+00,
       -5.98087715e+00, -2.97452457e+01, -5.37127914e+00, -2.96496041e+01,
        2.99980728e+01,  8.84271252e+00, -2.75567366e+01,  2.07062317e+01,
       -8.17572638e-01, -1.31242588e-01, -3.57214794e-01, -2.05674171e-02])
 message: 'Optimization terminated successfully'
    nfev: 3377
     nit: 148
    njev: 147
  status: 0
 success: True
       x: array([4.24058944e-02, 1.48959257e-03, 7.58007074e-06, 8.10609419e-03,
       1.08401802e-04, 4.14680911e-02, 5.54664377e-04, 6.27191650e-03,
       1.66928224e-02, 9.70920211e-08, 1.45657168e-13, 5.16239215e-05,
       4.92051181e-06, 3.09003245e-02, 3.25880984e-07, 3.34329456e-04,
       8.12888741e-01, 9.99350916e-01, 9.68944099e-01, 9.87595560e-01])]



 Best result found:

1.1842      fun: 1.1841630673762762
     jac: array(

In [7]:
#U(3) best set
x=set_vars(*[2.13534526e-02, 1.29459110e-02, 1.56385018e-03, 1.40332044e-02,
       3.79841225e-03, 2.31232638e-02, 6.26031188e-03, 3.75820684e-02,
       6.43040000e-02, 1.51362606e-04, 2.28404290e-08, 4.32263456e-03,
       6.86585763e-07, 8.97721071e-02, 3.51962675e-06, 5.81038727e-03,
       6.32852935e-01, 9.99993263e-01, 9.99993263e-01, 8.31420457e-01])

set_initial_constraints()
use_u3()
print("U(3)\t:",time(x),"\t",time(x)/search_space())

#U(2) best set
x=set_vars(*[3.77635125e-02, 4.10979013e-03, 8.29611055e-07, 1.20039511e-02,
       4.86194950e-05, 5.66727660e-02, 2.27013217e-04, 1.88579540e-21,
       3.19213050e-02, 1.08679115e-11, 8.43814786e-09, 7.79228338e-07,
       5.01570331e-10, 4.23513596e-02, 2.06955109e-05, 2.94022270e-10,
       6.26448319e-01, 9.99999996e-01, 8.74742725e-01, 4.01032151e-01])

set_initial_constraints()
use_u2()

print("U(2)\t:",time(x),"\t",time(x)/search_space())


#U(1) best set
x=set_vars(*[2.31851247e-02, 2.13406807e-04, 8.27351619e-13, 2.97435753e-03,
       6.03567373e-10, 6.49850958e-22, 5.33754386e-22, 3.83331933e-23,
       1.24678291e-02, 6.28195316e-17, 1.30569151e-12, 1.51327098e-06,
       5.65031509e-16, 3.18222786e-03, 5.17149487e-08, 7.91489614e-13,
       6.06782466e-01, 9.08166150e-01, 8.43732616e-01, 9.79122374e-01])

set_initial_constraints()
use_u1()

print("U(1)\t:",time(x),"\t",time(x)/search_space())



#B(3) best set
x=set_vars(*[4.23930482e-02, 1.48200330e-03, 8.64363856e-06, 8.08548052e-03,
       1.11010453e-04, 4.14778760e-02, 5.72605287e-04, 6.32202905e-03,
       1.66645494e-02, 5.68603056e-07, 3.54731414e-11, 5.59331286e-05,
       3.74880317e-10, 3.08997097e-02, 2.95948953e-08, 3.46037242e-04,
       8.12959828e-01, 9.98715621e-01, 9.69509278e-01, 9.95399462e-01])

set_initial_constraints()
use_b3()

print("B(3)\t:",time(x),"\t",time(x)/search_space())


#B(2) best set
x=set_vars(*[3.34095645e-02, 4.12334062e-04, 4.21959797e-08, 4.16956825e-03,
       8.66364032e-11, 3.27990046e-02, 9.96606788e-07, 2.46659400e-21,
       1.18372510e-02, 4.71876512e-08, 3.13629794e-09, 6.42020863e-08,
       3.56574998e-10, 1.49128910e-02, 2.40636338e-07, 3.54158884e-09,
       8.66412858e-01, 9.89778157e-01, 9.87323529e-01, 2.33401818e-01])

set_initial_constraints()
use_b2()

print("B(2)\t:",time(x),"\t",time(x)/search_space())

#B(1) best set
x=set_vars(*[1.98109800e-02, 8.04178674e-05, 6.47810741e-07, 1.31445593e-03,
       2.01001184e-07, 9.95892128e-25, 6.61744490e-34, 1.13225462e-26,
       8.22864305e-03, 4.78363385e-17, 2.64488644e-12, 3.57278533e-07,
       5.45612077e-09, 1.39037637e-03, 1.01186945e-08, 5.05536148e-19,
       8.31954697e-01, 9.65117145e-01, 8.73711686e-01, 9.27093372e-01])

set_initial_constraints()
use_b1()


print("B(1)\t:",time(x),"\t",time(x)/search_space())


U(3)	: 1.5048467477231724 	 0.5360372270351266
U(2)	: 1.2814118918511157 	 0.5518740630567535
U(1)	: 0.9296939372910756 	 0.5865715667519363
B(3)	: 1.183769788931055 	 0.5073236691481685
B(2)	: 1.0403012674304075 	 0.51230240114075
B(1)	: 0.8426811601710125 	 0.5617874401140083


In [16]:
use_b3()
print(1.3585/search_space())

0.5822071242079377
